In [1]:
from scipy import stats
import numpy as np

Getting 1000 samples of a normal distribution. Verifying they fall within that distribution

In [14]:
mean = 0 
sd = 1
X = stats.norm(mean, sd)
samples = X.rvs(100000)
a, b = np.percentile(samples, [2.5, 97.5])
range = b - a
if np.isclose(np.mean(samples), mean, atol=0.01):
    print("Sample mean is about equal to distribution mean")
if np.isclose(np.std(samples), sd, atol=0.01):
    print("Sample standard deviation is about equal to distribition std")
if np.isclose(range , 2*1.95, atol=0.1):
    print("Sample 95% range is close to the distribition IQR")

Sample mean is about equal to distribution mean
Sample standard deviation is about equal to distribition std
Sample 95% range is close to the distribition IQR
